# Ollama API 使用指南

## 回答补全 POST /api/generate

### 示例请求

- model: （必需）模型名称
- prompt: 要生成响应的提示
- suffix: 模型响应后的文本
- images: （可选）一个base64编码的图像列表（用于多模态模型，如 llava ）
- format: 返回响应的格式。目前唯一接受的值是 json
- options: 其他模型参数，如 temperature、seed 等
- system: 系统消息
- template: 要使用的提示模板
- context: 从先前对 /generate 的请求中返回的上下文参数，可以用于保持简短的对话记忆
- stream: 如果设置为 false ，响应将作为单个响应对象返回，而不是一系列对象流
- raw: 如果设置为 true ，将不会对提示进行任何格式化。如果您在请求API时指定了完整的模板提示，可以选择使用 raw 参数
- keep_alive: 控制模型在请求后保留在内存中的时间（默认：5m）

```
curl http://localhost:11434/api/generate -d '{
  "model": "llama3.2",
  "prompt": "为什么草是绿的？",
  "stream": false
}'
```

### 示例响应

- model: 模型名称
- created_at: 生成响应的时间
- context: 用于此响应的对话编码，可以在下一个请求中发送以保持对话记忆
- total_duration: 生成响应所花费的时间（纳秒）
- load_duration: 加载模型所花费的时间（纳秒）
- prompt_eval_count: 提示中的标记数量
- prompt_eval_duration: 评估提示所花费的时间（纳秒）
- eval_count: 响应中的标记数量
- eval_duration: 生成响应所花费的时间（纳秒）
- response: 如果响应是流式的则为空，如果不是流式的，这将包含完整的响应要计算响应生成速度（每秒生成的标记数，token/s）， 即eval_count / eval_duration * 10^9。
- done: 如果响应已完成，则为 true
- done_reason: 响应完成的原因

```json
{
  "model":"llama3.1",
  "created_at":"2024-08-08T02:54:10.819603411Z",
  "response":"",
  "done":true,
  "done_reason":"stop",
  "context":[1,2,3],
  "total_duration":8655401792,
  "load_duration":5924129727,
  "prompt_eval_count":17,
  "prompt_eval_duration":29196000,
  "eval_count":118,
  "eval_duration":2656329000
}
```

### JSON 模式

```
curl http://localhost:11434/api/generate -d '{
  "model": "llama3.2",
  "prompt": "为什么草是绿的？以JSON格式输出答案",
  "format": "json",
  "stream": false
}'
```

```json
{
  "model": "llama3.2",
  "created_at": "2025-03-21T12:21:01.387969Z",
  "response": "{\"reason\":\"因为植物中的叶绿素a、叶绿素b和叶绿素c\"}",
  "done": false
}
```

### 输入包含图片

要向多模态模型（如 llava 或 bakllava）提交图像，请提供 base64 编码的 images 列表


```bash
curl http://localhost:11434/api/generate -d '{
  "model": "llava",
  "prompt":"描述这张图片",
  "stream": false,
  "images": ["iVBORw0KGgoAAAANSUhEUgAAAG0AAABmCAYAAADBPx+VAAAACXBIWXMAAAsTAAALEwEAmpwYAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAA3VSURBVHgB7Z27r0zdG8fX743i1bi1ikMoFMQloXRpKFFIqI7LH4BEQ+NWIkjQuSWCRIEoULk0gsK1kCBI0IhrQVT7tz/7zZo888yz1r7MnDl7z5xvsjkzs2fP3uu71nNfa7lkAsm7d++Sffv2JbNmzUqcc8m0adOSzZs3Z+/XES4ZckAWJEGWPiCxjsQNLWmQsWjRIpMseaxcuTKpG/7HP27I8P79e7dq1ars/yL4/v27S0ejqwv+cUOGEGGpKHR37tzJCEpHV9tnT58+dXXCJDdECBE2Ojrqjh071hpNECjx4cMHVycM1Uhbv359B2F79+51586daxN/+pyRkRFXKyRDAqxEp4yMlDDzXG1NPnnyJKkThoK0VFd1ELZu3TrzXKxKfW7dMBQ6bcuWLW2v0VlHjx41z717927ba22U9APcw7Nnz1oGEPeL3m3p2mTAYYnFmMOMXybPPXv2bNIPpFZr1NHn4HMw0KRBjg9NuRw95s8PEcz/6DZELQd/09C9QGq5RsmSRybqkwHGjh07OsJSsYYm3ijPpyHzoiacg35MLdDSIS/O1yM778jOTwYUkKNHWUzUWaOsylE00MyI0fcnOwIdjvtNdW/HZwNLGg+sR1kMepSNJXmIwxBZiG8tDTpEZzKg0GItNsosY8USkxDhD0Rinuiko2gfL/RbiD2LZAjU9zKQJj8RDR0vJBR1/Phx9+PHj9Z7REF4nTZkxzX4LCXHrV271qXkBAPGfP/atWvu/PnzHe4C97F48eIsRLZ9+3a3f/9+87dwP1JxaF7/3r17ba+5l4EcaVo0lj3SBq5kGTJSQmLWMjgYNei2GPT1MuMqGTDEFHzeQSP2wi/jGnkmPJ/nhccs44jvDAxpVcxnq0F6eT8h4ni/iIWpR5lPyA6ETkNXoSukvpJAD3AsXLiwpZs49+fPn5ke4j10TqYvegSfn0OnafC+Tv9ooA/JPkgQysqQNBzagXY55nO/oa1F7qvIPWkRL12WRpMWUvpVDYmxAPehxWSe8ZEXL20sadYIozfmNch4QJPAfeJgW3rNsnzphBKNJM2KKODo1rVOMRYik5ETy3ix4qWNI81qAAirizgMIc+yhTytx0JWZuNI03qsrgWlGtwjoS9XwgUhWGyhUaRZZQNNIEwCiXD16tXcAHUs79co0vSD8rrJCIW98pzvxpAWyyo3HYwqS0+H0BjStClcZJT5coMm6D2LOF8TolGJtK9fvyZpyiC5ePFi9nc/oJU4eiEP0jVoAnHa9wyJycITMP78+eMeP37sXrx44d6+fdt6f82aNdkx1pg9e3Zb5W+RSRE+n+VjksQWifvVaTKFhn5O8my63K8Qabdv33b379/PiAP//vuvW7BggZszZ072/+TJk91YgkafPn166zXB1rQHFvouAWHq9z3SEevSUerqCn2/dDCeta2jxYbr69evk4MHDyY7d+7MjhMnTiTPnz9Pfv/+nfQT2ggpO2dMF8cghuoM7Ygj5iWCqRlGFml0QC/ftGmTmzt3rmsaKDsgBSPh0/8yPeLLBihLkOKJc0jp8H8vUzcxIA1k6QJ/c78tWEyj5P3o4u9+jywNPdJi5rAH9x0KHcl4Hg570eQp3+vHXGyrmEeigzQsQsjavXt38ujRo44LQuDDhw+TW7duRS1HGgMxhNXHgflaNTOsHyKvHK5Ijo2jbFjJBQK9YwFd6RVMzfgRBmEfP37suBBm/p49e1qjEP2mwTViNRo0VJWH1deMXcNK08uUjVUu7s/zRaL+oLNxz1bpANco4npUgX4G2eFbpDFyQoQxojBCpEGSytmOH8qrH5Q9vuzD6ofQylkCUmh8DBAr+q8JCyVNtWQIidKQE9wNtLSQnS4jDSsxNHogzFuQBw4cyM61UKVsjfr3ooBkPSqqQHesUPWVtzi9/vQi1T+rJj7WiTz4Pt/l3LxUkr5P2VYZaZ4URpsE+st/dujQoaBBYokbrz/8TJNQYLSonrPS9kUaSkPeZyj1AWSj+d+VBoy1pIWVNed8P0Ll/ee5HdGRhrHhR5GGN0r4LGZBaj8oFDJitBTJzIZgFcmU0Y8ytWMZMzJOaXUSrUs5RxKnrxmbb5YXO9VGUhtpXldhEUogFr3IzIsvlpmdosVcGVGXFWp2oU9kLFL3dEkSz6NHEY1sjSRdIuDFWEhd8KxFqsRi1uM/nz9/zpxnwlESONdg6dKlbsaMGS4EHFHtjFIDHwKOo46l4TxSuxgDzi+rE2jg+BaFruOX4HXa0Nnf1lwAPufZeF8/r6zD97WK2qFnGjBxTw5qNGPxT+5T/r7/7RawFC3j4vTp09koCxkeHjqbHJqArmH5UrFKKksnxrK7FuRIs8STfBZv+luugXZ2pR/pP9Ois4z+TiMzUUkUjD0iEi1fzX8GmXyuxUBRcaUfykV0YZnlJGKQpOiGB76x5GeWkWWJc3mOrK6S7xdND+W5N6XyaRgtWJFe13GkaZnKOsYqGdOVVVbGupsyA/l7emTLHi7vwTdirNEt0qxnzAvBFcnQF16xh/TMpUuXHDowhlA9vQVraQhkudRdzOnK+04ZSP3DUhVSP61YsaLtd/ks7ZgtPcXqPqEafHkdqa84X6aCeL7YWlv6edGFHb+ZFICPlljHhg0bKuk0CSvVznWsotRu433alNdFrqG45ejoaPCaUkWERpLXjzFL2Rpllp7PJU2a/v7Ab8N05/9t27Z16KUqoFGsxnI9EosS2niSYg9SpU6B4JgTrvVW1flt1sT+0ADIJU2maXzcUTraGCRaL1Wp9rUMk16PMom8QhruxzvZIegJjFU7LLCePfS8uaQdPny4jTTL0dbee5mYokQsXTIWNY46kuMbnt8Kmec+LGWtOVIl9cT1rCB0V8WqkjAsRwta93TbwNYoGKsUSChN44lgBNCoHLHzquYKrU6qZ8lolCIN0Rh6cP0Q3U6I6IXILYOQI513hJaSKAorFpuHXJNfVlpRtmYBk1Su1obZr5dnKAO+L10Hrj3WZW+E3qh6IszE37F6EB+68mGpvKm4eb9bFrlzrok7fvr0Kfv727dvWRmdVTJHw0qiiCUSZ6wCK+7XL/AcsgNyL74DQQ730sv78Su7+t/A36MdY0sW5o40ahslXr58aZ5HtZB8GH64m9EmMZ7FpYw4T6QnrZfgenrhFxaSiSGXtPnz57e9TkNZLvTjeqhr734CNtrK41L40sUQckmj1lGKQ0rC37x544r8eNXRpnVE3ZZY7zXo8NomiO0ZUCj2uHz58rbXoZ6gc0uA+F6ZeKS/jhRDUq8MKrTho9fEkihMmhxtBI1DxKFY9XLpVcSkfoi8JGnToZO5sU5aiDQIW716ddt7ZLYtMQlhECdBGXZZMWldY5BHm5xgAroWj4C0hbYkSc/jBmggIrXJWlZM6pSETsEPGqZOndr2uuuR5rF169a2HoHPdurUKZM4CO1WTPqaDaAd+GFGKdIQkxAn9RuEWcTRyN2KSUgiSgF5aWzPTeA/lN5rZubMmR2bE4SIC4nJoltgAV/dVefZm72AtctUCJU2CMJ327hxY9t7EHbkyJFseq+EJSY16RPo3Dkq1kkr7+q0bNmyDuLQcZBEPYmHVdOBiJyIlrRDq41YPWfXOxUysi5fvtyaj+2BpcnsUV/oSoEMOk2CQGlr4ckhBwaetBhjCwH0ZHtJROPJkyc7UjcYLDjmrH7ADTEBXFfOYmB0k9oYBOjJ8b4aOYSe7QkKcYhFlq3QYLQhSidNmtS2RATwy8YOM3EQJsUjKiaWZ+vZToUQgzhkHXudb/PW5YMHD9yZM2faPsMwoc7RciYJXbGuBqJ1UIGKKLv915jsvgtJxCZDubdXr165mzdvtr1Hz5LONA8jrUwKPqsmVesKa49S3Q4WxmRPUEYdTjgiUcfUwLx589ySJUva3oMkP6IYddq6HMS4o55xBJBUeRjzfa4Zdeg56QZ43LhxoyPo7Lf1kNt7oO8wWAbNwaYjIv5lhyS7kRf96dvm5Jah8vfvX3flyhX35cuX6HfzFHOToS1H4BenCaHvO8pr8iDuwoUL7tevX+b5ZdbBair0xkFIlFDlW4ZknEClsp/TzXyAKVOmmHWFVSbDNw1l1+4f90U6IY/q4V27dpnE9bJ+v87QEydjqx/UamVVPRG+mwkNTYN+9tjkwzEx+atCm/X9WvWtDtAb68Wy9LXa1UmvCDDIpPkyOQ5ZwSzJ4jMrvFcr0rSjOUh+GcT4LSg5ugkW1Io0/SCDQBojh0hPlaJdah+tkVYrnTZowP8iq1F1TgMBBauufyB33x1v+NWFYmT5KmppgHC+NkAgbmRkpD3yn9QIseXymoTQFGQmIOKTxiZIWpvAatenVqRVXf2nTrAWMsPnKrMZHz6bJq5jvce6QK8J1cQNgKxlJapMPdZSR64/UivS9NztpkVEdKcrs5alhhWP9NeqlfWopzhZScI6QxseegZRGeg5a8C3Re1Mfl1ScP36ddcUaMuv24iOJtz7sbUjTS4qBvKmstYJoUauiuD3k5qhyr7QdUHMeCgLa1Ear9NquemdXgmum4fvJ6w1lqsuDhNrg1qSpleJK7K3TF0Q2jSd94uSZ60kK1e3qyVpQK6PVWXp2/FC3mp6jBhKKOiY2h3gtUV64TWM6wDETRPLDfSakXmH3w8g9Jlug8ZtTt4kVF0kLUYYmCCtD/DrQ5YhMGbA9L3ucdjh0y8kOHW5gU/VEEmJTcL4Pz/f7mgoAbYkAAAAAElFTkSuQmCC"
  ]
}'
```

```json
{
  "model":"llava",
  "created_at":"2024-08-08T07:33:55.481713465Z",
  "response":" The image shows a cartoon of an animated character that resembles a cute pig with large eyes and a smiling face. It appears to be in motion, indicated by the lines extending from its arms and tail, giving it a dynamic feel as if it is waving or dancing. The style of the image is playful and simplistic, typical of line art or stickers. The character's design has been stylized with exaggerated features such as large ears and a smiling expression, which adds to its charm. ",
  "done":true,
  "done_reason":"stop",
  "context":[1,2,3],
  "total_duration":2960501550,
  "load_duration":4566012,
  "prompt_eval_count":1,
  "prompt_eval_duration":758437000,
  "eval_count":108,
  "eval_duration":2148818000
}
```

### 可复现输出

将 seed 设置为一个固定数值，得到可复现的输出：

```bash
curl http://localhost:11434/api/generate -d '{
  "model": "llama3.2",
  "prompt": "为什么草是绿的？",
  "stream": false,
  "options": {
    "seed": 1001
  }
}'
```

## 对话补全 POST /api/chat

使用指定模型生成聊天中的下一条消息。这也是一个流式端点，因此会有一系列响应。


- model：（必需）模型名称
- messages：聊天消息，这可以用于保持聊天记忆
- tools：模型支持使用的工具。需要将 stream 设置为 false
- message 对象具有以下字段：
    - role：消息的角色，可以是 system、user、assistant 或 tool
    - content：消息的内容
    - images（可选）：要包含在消息中的图像列表（对于如 llava 之类的多模态模型）
    - tool_calls（可选）：模型想要使用的工具列表
- format：返回响应的格式。目前唯一接受的值是 json
- options：其他模型参数，如 temperature、seed 等
- stream：如果为 false，响应将作为单个响应对象返回，而不是一系列对象流
- keep_alive：控制请求后模型在内存中保持加载的时间（默认：5m）


### 示例请求

```bash
curl http://localhost:11434/api/chat -d '{
  "model": "llama3.2",
  "messages": [
    {
      "role": "user",
      "content": "为什么草是绿的？"
    }
  ],
  "stream": false
}'
```

### 示例响应

```json
{
  "model":"llama3.2",
  "created_at":"2024-08-08T03:54:37.187621765Z",
  "message":{
    "role":"assistant",
    "content":""
  },
  "done_reason":"stop",
  "done":true,
  "total_duration":5730533217,
  "load_duration":5370535786,
  "prompt_eval_count":17,
  "prompt_eval_duration":29621000,
  "eval_count":13,
  "eval_duration":273810000
}
```

### 带历史记录

发送带有对话历史记录的聊天消息。可以使用相同的方法开始多轮对话或思维链提示。



```bash
curl http://localhost:11434/api/chat -d '{
  "model": "llama3.2",
  "messages": [
    {
      "role": "user",
      "content": "为什么草是绿色的？"
    },
    {
      "role": "assistant",
      "content": "因为草里面含有叶绿素。"
    },
    {
      "role": "user",
      "content": "为什么叶绿素让草看起来是绿色的？"
    }
  ],
  "stream": false
}'
```

```json
{
  "model": "llama3.2",
  "created_at": "2025-03-21T12:30:06.162708Z",
  "message": {
    "role": "assistant",
    "content": "这是一个科学故事！\n\n叶绿素是一种特殊的化合物，它存在于植物细胞中，特别是在叶片和其他颜色不够明显的部分。这种化合物通过吸收光线中的蓝色、黄色和红色波段，并反射出绿色波段，使草看起来是绿色的。\n\n当阳光照射在植物上时，光能传递给叶细胞，这些细胞中含有叶绿素。当光能与叶绿素结合时，叶绿素会吸收能量并释放出绿色光。这种作用使得叶片能够反射出绿色的波段，使其看起来是绿色的。\n\n然而，如果阳光照射在植物上太强，也可能会导致叶绿素被破坏或变性，从而改变草的颜色。这就是为什么在阴天、冬季或受到过度照明的情况下，grass或其他植物变得更暗或变成棕色的一种原因。\n\n所以，这个故事告诉我们，叶绿素和光能之间的关系使得植物能够反映出绿色的波段，从而让它们看起来是绿色的！"
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 5138881917,
  "load_duration": 34037458,
  "prompt_eval_count": 62,
  "prompt_eval_duration": 121000000,
  "eval_count": 251,
  "eval_duration": 4981000000
}
```



## 创建模型 POST /api/create

- name：要创建的模型名称
- modelfile（可选）：Modelfile 的内容
- stream（可选）：如果为 false，响应将作为单个响应对象返回，而不是对象流
- path（可选）：Modelfile 的路径

```
curl http://localhost:11434/api/create -d '{
  "name": "mario",
  "modelfile": "FROM llama3\nSYSTEM You are mario from Super Mario Bros."
}'
```



## 复制模型 POST /api/copy

复制一个模型，使用另一个名称复制已有模型。

```bash
curl http://localhost:11434/api/copy -d '{
  "source": "llama3.2",
  "destination": "llama3-backup"
}'
```

## 删除模型 DELETE /api/delete


删除模型及其数据。


```bash
curl -X DELETE http://localhost:11434/api/delete -d '{
  "name": "llama3.1"
}'
```

## 列出运行模型 GET /api/ps

列出当前加载到内存中的模型。

```bash
curl http://localhost:11434/api/ps
``` 

## 列出本地模型 GET /api/tags

列出本地可用的模型。


```bash
curl http://localhost:11434/api/tags
```


## 显示模型信息 POST /api/show


显示有关模型的信息，包括详细信息、modelfile、模板、参数、许可证、系统提示。


- name：要显示的模型名称
- verbose（可选）：如果设置为 true，则返回详细响应字段的完整数据


```bash
curl http://localhost:11434/api/show -d '{
  "name": "llama3.2",
  "verbose": false
}'
````


## 拉取模型 POST /api/pull


从 ollama 库下载模型。中断的拉取操作会从断点继续下载，多个调用将共享相同的下载进度。


- name：要拉取的模型名称
- insecure（可选）：允许对库进行不安全连接。建议仅在开发期间，从自己的库中拉取时使用此选项。
- stream（可选）：如果为 false，响应将作为单个响应对象返回，而不是对象流。

```
curl http://localhost:11434/api/pull -d '{
  "name": "llama3.2"
}'
```


## 推送模型 POST /api/push


将模型上传到模型库。需要先注册 ollama.ai 并添加公钥。



- name：要推送的模型名称，格式为 <namespace>/<model>:<tag>
- insecure（可选）：允许对库进行不安全连接。仅在开发期间推送到自己的库时使用此选项。
- stream（可选）：如果为 false，响应将作为单个响应对象返回，而不是对象流。


```bash
curl http://localhost:11434/api/push -d '{
  "name": "mattw/pygmalion:latest"
}'
```

## 生成嵌入 POST /api/embed


从模型生成嵌入。

- model：要从中生成嵌入的模型名称
- input：要生成嵌入的文本或文本列表
- truncate：截断每个输入的末尾以适应上下文长度。如果为 false 并且超过上下文长度，则返回错误。默认值为 true
- options：其他模型参数，如 temperature、seed 等
- keep_alive：控制请求后模型在内存中保持加载的时间（默认：5m）

```
curl http://localhost:11434/api/embed -d '{
  "model": "llama3.2",
  "input": "为什么草是绿的？"
}'

{
  "model":"llama3.2",
  "embeddings":[[
    -0.008059342,-0.013182715,0.019781841,0.012018124,-0.024847334,
    -0.0031902494,-0.02714767,0.015282277,0.060032737,...
  ]],
  "total_duration":3041671009,
  "load_duration":2864335471,
  "prompt_eval_count":7
}
```

## 错误处理

- 400 Bad Request：请求格式错误。
- 404 Not Found：请求的资源不存在。
- 500 Internal Server Error：服务器内部错误。